In [3]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations
from itertools import cycle  # For creating iterators that cycle through a sequence
from lifelines import CoxTimeVaryingFitter  # For fitting time-varying Cox proportional hazards models
from sklearn.preprocessing import StandardScaler  # For standardizing data features
from scipy.stats import shapiro  # For conducting Shapiro-Wilk tests for normality of data
import glob  # For finding all files matching a specified pattern
import logging  # For logging information and debugging
import sys  # For system-specific parameters and functions
import seaborn as sns  # For data visualization, especially for statistical graphics
import matplotlib.pyplot as plt  # For creating plots and figures

In [7]:
# Define the number of years, weeks per year, and number of health regions
num_years = 8
weeks_per_year = 52
num_regions = 7

# Create a date range for 8 years (from 2015 to 2022), with weekly frequency
dates = pd.date_range(start='2015-01-01', periods=num_years * weeks_per_year, freq='W-MON')

# Generate sample data for each health region and each week
data = {
    'DateWeek': np.tile(dates, num_regions),  # Repeating date range for each region
    'Year': np.tile(dates.year, num_regions),  # Repeating years for each region
    'WeekNum': np.tile(range(1, weeks_per_year + 1), num_years * num_regions),  # Weekly numbers for each year and region
    'HealthReg': np.repeat(range(1, num_regions + 1), num_years * weeks_per_year),  # Health region identifiers
    'HealthRegPopulation': np.repeat([np.random.randint(300000, 500000) for _ in range(num_regions)], num_years * weeks_per_year),  # Population of each health region
    'MalePopulation': np.repeat([np.random.randint(200000, 300000) for _ in range(num_regions)], num_years * weeks_per_year),  # Male population in each region
    'FemalePopulation': np.repeat([np.random.randint(100000, 200000) for _ in range(num_regions)], num_years * weeks_per_year),  # Female population in each region
    'Sex': np.random.choice([0, 1], num_years * weeks_per_year * num_regions),  # Randomly assigning sex (0 for female, 1 for male)
    'AdmissType': np.random.choice([1, 2, 3], num_years * weeks_per_year * num_regions),  # Randomly assigning admission types
    'Hospitalisations': np.random.randint(0, 100, num_years * weeks_per_year * num_regions),  # Number of hospitalizations
    'Event': np.random.choice([0, 1], num_years * weeks_per_year * num_regions),  # Binary event indicator (e.g., an occurrence of a certain event)
    'WeeklyMaxAQHI': np.random.uniform(0, 10, num_years * weeks_per_year * num_regions),  # Weekly maximum Air Quality Health Index (AQHI)
    'Baseline': np.random.choice([0, 1], num_years * weeks_per_year * num_regions)  # Binary baseline indicator
}

# Create DataFrame from the generated data
synthetic_df = pd.DataFrame(data)


In [4]:
# Initialize the rate columns with zeros
synthetic_df["MaleHospitalizationRate"] = 0.0
synthetic_df["FemaleHospitalizationRate"] = 0.0

# Calculate rates of hospitalization per 100 inhabitants for each sex
synthetic_df.loc[synthetic_df["Sex"] == 0, "MaleHospitalizationRate"] = (synthetic_df["Hospitalisations"] / synthetic_df["MalePopulation"]) * 100
synthetic_df.loc[synthetic_df["Sex"] == 1, "FemaleHospitalizationRate"] = (synthetic_df["Hospitalisations"] / synthetic_df["FemalePopulation"]) * 100

In [5]:
# Sort the dataframe by HealthReg and DateWeek to ensure proper ordering
synthetic_df = synthetic_df.sort_values(['HealthReg', 'DateWeek'])

In [ ]:
# Define lists for health regions, admission types, and sex
reg = [1, 2, 3, 4, 5, 6, 7]
admission = [1, 2, 3, 4, 5]
sex = [0, 1]

# Loop through each combination of health region, admission type, and sex
for x in reg:
    for y in admission:
        for z in sex:
            # Filter the DataFrame for the current region, admission type, and sex
            group = synthetic_df[(synthetic_df['HealthReg'] == x) & 
                                 (synthetic_df['AdmissType'] == y) & 
                                 (synthetic_df['Sex'] == z)]

            # Choose the correct column based on sex
            if z == 1:
                health_admission_col = 'FemaleHospitalizationRate'
            else:
                health_admission_col = 'MaleHospitalizationRate'

            # Calculate the threshold for the top 10% of health admissions
            threshold = group[health_admission_col].quantile(0.9)
            print(f'Region: {x}, Admission Type: {y}, Sex: {z}, Threshold: {threshold}')

            # Assign Baseline2 and Event2 based on the top 10% of health admissions
            group['Baseline2'] = group[health_admission_col].apply(lambda val: 1 if val >= threshold else 0)
            group['Event2'] = group['Baseline2']  # Event2 is the same as Baseline2

            # Update the original DataFrame with the new columns
            synthetic_df.loc[group.index, 'Baseline2'] = group['Baseline2']
            synthetic_df.loc[group.index, 'Event2'] = group['Event2']

In [8]:
# Convert 'DateWeek' column to datetime type if it's not already
synthetic_df['DateWeek'] = pd.to_datetime(synthetic_df['DateWeek'])

# Function to check for lag between AQHI event week and increase in health admissions
def check_lag(df, health_admission_col):
    lags = []
    event_weeks = df[df['Event2'] == 1]['DateWeek']  # Identify weeks with an AQHI event

    for event_week in event_weeks:
        # Get the weeks following the AQHI event week
        following_weeks = df[df['DateWeek'] > event_week]
        
        # Find the first week with an increase in health admissions after the event week
        for i, row in following_weeks.iterrows():
            if row[health_admission_col] >= df[health_admission_col].quantile(0.9):
                lag = (row['DateWeek'] - event_week).days // 7  # Calculate lag in weeks
                lags.append(lag)
                break

    return lags

# Initialize dictionary to store lags for different groups
lags_dict = {'HealthReg': [], 'AdmissType': [], 'Sex': [], 'LagInWeeks': []}

# Iterate over regions, admission types, and sexes
for x in reg:
    for y in admission:
        for z in sex:
            # Filter the dataset for the specific group
            group = synthetic_df[(synthetic_df['HealthReg'] == x) & 
                                 (synthetic_df['AdmissType'] == y) & 
                                 (synthetic_df['Sex'] == z)]

            # Choose the correct column for health admissions based on sex
            if z == 1:
                health_admission_col = 'FemaleHospitalizationRate'
            else:
                health_admission_col = 'MaleHospitalizationRate'

            # Check for lag in the group
            lags = check_lag(group, health_admission_col)

            # Store results in the dictionary
            for lag in lags:
                lags_dict['HealthReg'].append(x)
                lags_dict['AdmissType'].append(y)
                lags_dict['Sex'].append(z)
                lags_dict['LagInWeeks'].append(lag)

# Convert the dictionary to a DataFrame for easier analysis
lags_df = pd.DataFrame(lags_dict)

# Plot histogram of lag in weeks
lags_df.hist('LagInWeeks')

In [ ]:
# Define a function to check for lag between AQHI event week and an increase in health admissions
def check_lag(df, airindex_col):
    lags = []
    event_weeks = df[df['Event2'] == 1]['DateWeek']

    # Iterate over each event week where an event occurred
    for event_week in event_weeks:
        # Get the weeks prior to the AQHI event week, sorted in descending order
        previous_weeks = df[df['DateWeek'] < event_week].sort_values(by='DateWeek', ascending=False)

        # Find the first week with AQHI >= 7 before the event week
        for i, row in previous_weeks.iterrows():
            if row[airindex_col] >= 7:
                # Calculate the lag in weeks between the event and the AQHI week
                lag = (event_week - row['DateWeek']).days // 7
                lags.append({
                    'EventWeek': event_week,
                    'AQHIWeek': row['DateWeek'],
                    'LagInWeeks': lag
                })
                break  # Stop looking once the first lag is found

    return lags

# Initialize a dictionary to store lag data for different groups
lags_dict = {'HealthReg': [], 'AdmissType': [], 'Sex': [], 'EventWeek': [], 'AQHIWeek': [], 'LagInWeeks': []}

# Iterate over each combination of health region, admission type, and sex
for x in reg:
    for y in admission:
        for z in sex:
            group = synthetic_df[(synthetic_df['HealthReg'] == x) &
                                 (synthetic_df['AdmissType'] == y) &
                                 (synthetic_df['Sex'] == z)]

            # The column representing AQHI levels
            airindex_col = 'WeeklyMaxAQHI'

            # Check for lags in the current group
            lags = check_lag(group, airindex_col)

            # Append the results to the dictionary
            for lag in lags:
                lags_dict['HealthReg'].append(x)
                lags_dict['AdmissType'].append(y)
                lags_dict['Sex'].append(z)
                lags_dict['EventWeek'].append(lag['EventWeek'])
                lags_dict['AQHIWeek'].append(lag['AQHIWeek'])
                lags_dict['LagInWeeks'].append(lag['LagInWeeks'])

# Convert the dictionary into a DataFrame for easier analysis
lags_df = pd.DataFrame(lags_dict)

# Add a column to indicate whether the lag is within 5 weeks
lags_df['LagCondition'] = lags_df['LagInWeeks'].apply(lambda x: 1 if x <= 5 else 0)

# Merge the lag information back into the original dataset
synthetic_df = pd.merge(
    synthetic_df,
    lags_df[['HealthReg', 'AdmissType', 'Sex', 'EventWeek', 'LagCondition']],
    how='left',
    left_on=['HealthReg', 'AdmissType', 'Sex', 'DateWeek'],
    right_on=['HealthReg', 'AdmissType', 'Sex', 'EventWeek']
)

# Fill NaN values with 0 for rows where the condition is not met
synthetic_df['LagCondition'] = synthetic_df['LagCondition'].fillna(0).astype(int)

In [ ]:
# Creating new columns 'Baseline3' and 'Event3' by copying the values from the 'LagCondition' column
synthetic_df['Baseline3'] = synthetic_df['LagCondition']
synthetic_df['Event3'] = synthetic_df['LagCondition']

In [ ]:
# Sample regions, admission types, and sex categories
reg = [1, 2, 3, 4, 5, 6, 7]
admission = [1, 2, 3, 4, 5]
sex = [0, 1]

# Loop through each region, admission type, and sex category
for x in reg:
    for y in admission:
        for z in sex:
            # Filter the dataframe based on the current region, admission type, and sex
            group = synthetic_df[(synthetic_df['HealthReg'] == x) & 
                                 (synthetic_df['AdmissType'] == y) & 
                                 (synthetic_df['Sex'] == z)]
            
            # Separate the group into baselines and controls
            baselines = group[group['Baseline3'] == 1.0]
            controls = group[group['Baseline3'] == 0.0]

            print(f"\nProcessing HealthReg: {group['HealthReg'].iloc[0]}")
            print(f"Processing AdmissType: {group['AdmissType'].iloc[0]}")
            print(f"Processing Sex: {group['Sex'].iloc[0]}")
            print(f"Baselines in this HealthReg: {len(baselines)}")
            print(f"Potential controls in this HealthReg: {len(controls)}")

            # Group baselines into batches of 1 consecutive weeks
            baselines['BatchID'] = (baselines['WeekNum'] != baselines['WeekNum'].shift() + 1).cumsum()
            baseline_batches = baselines.groupby('BatchID')

            control_cases = []
            matching_id = 1

            # Process each batch of baselines
            for batch_id, batch in baseline_batches:
                print(f"\nProcessing Batch ID: {batch_id}")
                print(f"Batch size: {len(batch)}")

                if len(batch) < 4:
                    print("Skipping incomplete batch")
                    continue  # Skip incomplete batches

                # Extract baseline year and weeks for the batch
                baseline_year = batch['Year'].iloc[0]
                baseline_weeks = batch['WeekNum'].tolist()
                print(f"Baseline Year: {baseline_year}")
                print(f"Baseline Weeks: {baseline_weeks}")

                # Identify potential control years excluding the baseline year
                control_years = controls['Year'].unique()
                control_years = control_years[control_years != baseline_year]
                print(f"Potential Control Years: {control_years}")

                if len(control_years) < 7:
                    print("Not enough control years available. Skipping batch.")
                    continue  # Not enough control years available

                # Cycle through control years and weeks
                week_cycle = cycle(baseline_weeks)

                for year in control_years:
                    for _ in baseline_weeks:
                        week = next(week_cycle)
                        control_case = controls[(controls['Year'] == year) & (controls['WeekNum'] == week)]
                        print('Control year', year)
                        print('Control week', week)

                        if not control_case.empty:
                            control_case = control_case.iloc[0].copy()
                            control_case['MatchingID'] = matching_id
                            control_cases.append(control_case)
                            print(f"Matched: Baseline Year {baseline_year}, Week {week} with Control Year {year}, MatchID {matching_id}")
                        else:
                            print(f"No match found for Baseline Year {baseline_year}, Week {week}")

                # Assign MatchingID to baseline batch
                batch['MatchingID'] = matching_id
                control_cases.extend(batch.to_dict('records'))

                print(f"Assigned MatchingID {matching_id} to batch")

                matching_id += 1

                # Convert all elements to dictionaries
                data_dicts = [item.to_dict() if isinstance(item, pd.Series) else item for item in control_cases]

                # Create DataFrame from list of dictionaries
                result_df = pd.DataFrame(data_dicts)

                print(f"Region: {x}, Admission: {y}, Sex {z}, Total rows: {len(result_df)}")
                print(f"Baseline cases: {result_df['Baseline3'].sum()}")
                print(f"Control cases: {len(result_df) - result_df['Baseline2'].sum()}")

                # Save the result to an Excel file
                result_df.to_excel(f'ControlCases_Reg_{x}_Adm_{y}_Sex_{z}.xlsx', index=False)
                print(f'Saved: ControlCases_Reg_{x}_Adm_{y}_Sex_{z}.xlsx')

In [ ]:
def filter_control(file_path):
    """
    Read and filter data from an Excel file.
    
    Parameters:
    file_path (str): The path to the Excel file.
    
    Returns:
    pd.DataFrame: The filtered DataFrame.
    """
    df_filtered = pd.read_excel(file_path)
    return df_filtered

# Path to the directory containing the saved Excel files
file_paths = glob.glob('ControlCases_Reg_*.xlsx')

# Create an Excel writer to save the results in a single file
with pd.ExcelWriter('filtered_datasets.xlsx', engine='xlsxwriter') as writer:
    # Process each file in the directory
    for file_path in file_paths:
        # Extract the sheet name from the file name by removing directory path and file extension
        sheet_name = file_path.split('/')[-1].replace('ControlCases_', '').replace('.xlsx', '')
        
        # Filter the data using the filter_control function
        df_filtered = filter_control(file_path)
        
        # Save the filtered DataFrame to the Excel file with the extracted sheet name
        df_filtered.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f'Saved sheet {sheet_name}')


In [ ]:
# Define the function to perform correlation and normality tests on a given file
def analyze_file(file_path):
    df = pd.read_excel(file_path)  # Read data from the Excel file
    df.fillna(0, inplace=True)  # Replace NaN values with 0 for consistency
    df['WeeklyMaxAQHI'] += 1e-6  # Small adjustment to avoid log(0)
    df['WeeklyMaxAQHI'] = np.log(df['WeeklyMaxAQHI'])  # Apply log transformation for normalization
    
    # Calculate the correlation matrix for the specified columns
    correlation_matrix = df[["WeeklyMaxAQHI", "Baseline3"]].corr()
    
    # Perform Shapiro-Wilk test for normality on the 'WeeklyMaxAQHI' column
    normality_results = df[["WeeklyMaxAQHI"]].apply(lambda x: shapiro(x)[1])
    normality_results = normality_results.to_frame(name='p_value')  # Convert results to DataFrame
    
    return correlation_matrix, normality_results

# Path to the directory containing the saved Excel files
file_paths = glob.glob('ControlCases_Reg_*.xlsx')

# Create an Excel writer to save the results in a new file
writer = pd.ExcelWriter('Correlation_Results_With_Normality.xlsx', engine='xlsxwriter')

# Process each Excel file in the specified directory
for file_path in file_paths:
    # Extract sheet name from the file name
    sheet_name = file_path.split('/')[-1].replace('ControlCases_', '').replace('.xlsx', '')
    
    # Analyze the file to obtain correlation and normality test results
    correlation_matrix, normality_results = analyze_file(file_path)
    
    # Save the correlation matrix and normality results to the Excel writer
    correlation_matrix.to_excel(writer, sheet_name=sheet_name)
    normality_results.to_excel(writer, sheet_name=sheet_name, startrow=correlation_matrix.shape[0] + 2)

    print(f'Saved sheet {file_path}')

# Finalize and save the Excel file with all results
writer.close()

In [ ]:
# Define the function to perform the grouping and aggregation
def analyze_sheet(df):
    # Group by 'MatchingID', 'Baseline3', and 'Year' and calculate the mean of 'MalePopulation' and 'FemalePopulation'
    mean_population = df.groupby(['MatchingID', 'Baseline3', 'Year'])[['MalePopulation', 'FemalePopulation']].mean()

    # Group by 'MatchingID', 'Baseline3', and 'Year' and calculate the sum of 'Hospitalisations'
    sum_hospitalization = df.groupby(['MatchingID', 'Baseline3', 'Year'])['Hospitalisations'].sum()

    # Group by 'MatchingID', 'Baseline3', and 'Year' and calculate the max of 'WeeklyMaxAQHI'
    max_weekly_max_aqhi = df.groupby(['MatchingID', 'Baseline3', 'Year'])['WeeklyMaxAQHI'].max()

    # Group by 'MatchingID', 'Baseline3', and 'Year' and calculate the max of 'Event3'
    max_event = df.groupby(['MatchingID', 'Baseline3', 'Year'])['Event3'].max()

    # Define a fixed duration variable (e.g., 35 days)
    duration = 35

    # Combine the results into a single DataFrame
    result = pd.DataFrame({
        'MeanMalePopulation': mean_population['MalePopulation'],
        'MeanFemalePopulation': mean_population['FemalePopulation'],
        'SumHospitalization': sum_hospitalization,
        'MaxWeeklyMaxAQHI': max_weekly_max_aqhi,
        'MaxEvent': max_event,
        'Duration': duration
    }).reset_index()

    # Calculate hospitalization rate per 100 individuals in the female population
    result['HospRate'] = result['SumHospitalization'] / result['MeanFemalePopulation'] * 100

    return result

# Read the filtered datasets from an Excel file with multiple sheets
file_path = 'filtered_datasets.xlsx'
excel_file = pd.ExcelFile(file_path)

# Create an Excel writer to save the aggregated results
with pd.ExcelWriter('Aggregated_Results.xlsx', engine='xlsxwriter') as writer:
    # Process each sheet in the Excel file
    for sheet_name in excel_file.sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(excel_file, sheet_name=sheet_name)
        
        # Analyze the DataFrame using the defined function
        result = analyze_sheet(df)
        
        # Save the results to the new Excel file
        result.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f'Saved sheet {sheet_name}')

In [ ]:
# Define the function to perform data preprocessing and Cox model fitting
def analyze_sheet(df):
    # Fill missing values with 0 and perform necessary transformations
    df.fillna(0, inplace=True)
    df['MaxWeeklyMaxAQHI'] += 1e-6  # Avoid log(0) by adding a small constant
    df['MaxWeeklyMaxAQHI'] = np.log(df['MaxWeeklyMaxAQHI'])  # Log transform for AQHI
    df['HospRate'] += 1e-6  # Avoid issues with zero rates
    df['AQHI_Baseline_Interaction'] = df['MaxWeeklyMaxAQHI'] * df['Baseline3']  # Interaction term
    
    # Select relevant columns for the analysis
    selected_weeks_df = df[['MaxWeeklyMaxAQHI', "Baseline3", 'Duration', 'MaxEvent', 'AQHI_Baseline_Interaction']]

    # Fit the Cox Proportional Hazards model
    cph = CoxPHFitter(penalizer=0.1)  # Initialize model with a penalizer to handle multicollinearity
    cph.fit(selected_weeks_df, duration_col='Duration', event_col='MaxEvent', show_progress=True)  # Fit the model

    # Extract the summary of the model fit
    summary = cph.summary
    
    return summary

# Define the file path for the Excel file containing filtered datasets
file_path = 'Aggregated_Results.xlsx'
excel_file = pd.ExcelFile(file_path)  # Load the Excel file

# Create an Excel writer object to save the model results
with pd.ExcelWriter('CoxPH_Results.xlsx', engine='xlsxwriter') as writer:
    # Iterate through each sheet in the Excel file
    for sheet_name in excel_file.sheet_names:
        # Load the data from the current sheet into a DataFrame
        df = pd.read_excel(excel_file, sheet_name=sheet_name)
        
        # Apply the analysis function to the DataFrame
        summary = analyze_sheet(df)
        
        # Save the analysis results to a new Excel file
        summary.to_excel(writer, sheet_name=sheet_name, index=True)

        print(f'Saved sheet {sheet_name}')  # Confirmation message